## Definition

**Given**: `m` items and their weight wi and value vi, knapsack
with weight limit `b`.

**Find**: What is the most valuable subset of items that will fit
into the knapsack?

### Variables

$ z = (z_1, z_2, ... z_m) $

In [283]:
import numpy as np, operator

M = 8 # number of items within bag
I = np.random.randint(0,5,M)
V = np.random.rand(M) # value of each item
W = np.random.rand(M) # weight of each item
T = np.random.rand() * (M)
X = np.random.randint(0,2,M)
print(W)
def print_information(M=M,V=V,W=W,T=T):
    for i, value in enumerate(V):
        print("Item number {0} has a value of {1} and a weight of {2}".format(i, np.round(value,2), np.round(W[i],2)))
    print("The total maximum weight of your bag is {0}".format(round(T,2)))

def calculate_value(X, W=W, V=V, T=T):
    
    # introduce uncertainty
    V += np.array(np.random.normal(0,0.01))
    W += np.array(np.random.normal(0,0.01))
    print(V)
    print(W)
    return (np.dot(X, W) < T) * np.dot(X, V)

def switch(boolean):
    return not boolean

print_information()
print(X)
print(np.dot(X, V))
print(np.dot(X, W))
calculate_value(X)

[ 0.96526411  0.82413376  0.50477149  0.49195509  0.24034506  0.82251793
  0.32406173  0.90981745]
Item number 0 has a value of 0.8 and a weight of 0.97
Item number 1 has a value of 0.0 and a weight of 0.82
Item number 2 has a value of 0.66 and a weight of 0.5
Item number 3 has a value of 0.97 and a weight of 0.49
Item number 4 has a value of 0.33 and a weight of 0.24
Item number 5 has a value of 0.86 and a weight of 0.82
Item number 6 has a value of 0.37 and a weight of 0.32
Item number 7 has a value of 0.29 and a weight of 0.91
The total maximum weight of your bag is 7.81
[1 0 1 1 1 0 1 0]
3.12073680082
2.52639747565
[ 0.80882981  0.01232811  0.66443843  0.97769885  0.33441718  0.86785312
  0.37372394  0.29934516]
[ 0.96554721  0.82441686  0.50505459  0.49223819  0.24062816  0.82280103
  0.32434483  0.91010055]


3.1591082145327234

In [279]:
def mcmc(iterations=100000, M=M, I=I, V=V, W=W, T=T, X=X, burn_period=1000):
    
    acceptance_total = 0
    value = 0
    X_history = np.zeros(shape=(iterations,M))
    frequency = dict()

    for i in range(iterations):
        
        
        if i % 1000 == 0:
            print("{0}% complete...".format(round((i+1) / iterations * 100.0,2)))
        
        if value == 0:
            value = 1e-10
            
        U = np.random.uniform()
        
        # save previous iteration's values in case we do not accept
        old_value = value
        old_X = X.copy()
        
        # propose a new configuration for X
        # pick a random item from the M items, and permute it
        idx = np.random.randint(0,M)
        X[idx] = switch(X[idx]) 
        
        # calculate the value of the proposed configuraiton
        value = calculate_value(X, W, V, T)        
        acceptance_ratio = min(1, value / old_value)
        acceptance_total += acceptance_ratio
        
        if acceptance_ratio  <= U:
            X = old_X
        
        if str(X) not in frequency:
            frequency[str(X)] = 1
        else:
            frequency[str(X)] += 1
            
        X_history[i,:] = X

    X_history = X_history[burn_period:,:]
    
    return X_history, sorted(frequency.items(), key=operator.itemgetter(1), reverse=True), acceptance_total / iterations

In [280]:
X_history, frequency, average_acceptance = mcmc(50000, burn_period=10000)

0.0% complete...
2.0% complete...
4.0% complete...
6.0% complete...
8.0% complete...
10.0% complete...
12.0% complete...
14.0% complete...
16.0% complete...
18.0% complete...
20.0% complete...
22.0% complete...
24.0% complete...
26.0% complete...
28.0% complete...
30.0% complete...
32.0% complete...
34.0% complete...
36.0% complete...
38.0% complete...
40.0% complete...
42.0% complete...
44.0% complete...
46.0% complete...
48.0% complete...
50.0% complete...
52.0% complete...
54.0% complete...
56.0% complete...
58.0% complete...
60.0% complete...
62.0% complete...
64.0% complete...
66.0% complete...
68.0% complete...
70.0% complete...
72.0% complete...
74.0% complete...
76.0% complete...
78.0% complete...
80.0% complete...
82.0% complete...
84.0% complete...
86.0% complete...
88.0% complete...
90.0% complete...
92.0% complete...
94.0% complete...
96.0% complete...
98.0% complete...


In [281]:
frequency

[('[1 0 0 0 0 0 0 0]', 23322),
 ('[0 0 0 0 1 0 0 0]', 2181),
 ('[0 0 0 0 0 0 1 0]', 1965),
 ('[0 1 0 0 0 0 0 0]', 1344),
 ('[0 0 1 0 0 0 0 0]', 727),
 ('[0 0 0 0 0 0 0 1]', 406),
 ('[0 0 0 0 0 1 0 0]', 241),
 ('[1 0 0 0 0 0 1 0]', 215),
 ('[0 0 1 0 1 0 0 0]', 204),
 ('[0 0 0 0 1 1 0 0]', 184),
 ('[1 0 0 0 1 0 0 0]', 182),
 ('[1 0 1 0 0 0 0 0]', 174),
 ('[0 0 0 0 1 0 0 1]', 169),
 ('[0 0 0 0 1 0 1 0]', 165),
 ('[1 0 0 0 0 1 0 0]', 153),
 ('[0 1 0 0 1 0 0 0]', 152),
 ('[0 0 0 0 0 0 1 1]', 150),
 ('[0 0 1 0 0 0 1 0]', 148),
 ('[1 1 0 0 0 0 0 0]', 143),
 ('[0 0 0 0 0 1 1 0]', 137),
 ('[0 0 1 0 0 0 0 1]', 132),
 ('[0 1 0 0 0 0 1 0]', 127),
 ('[1 0 0 0 0 0 0 1]', 125),
 ('[0 0 1 0 0 1 0 0]', 117),
 ('[1 0 0 1 0 0 0 0]', 116),
 ('[1 1 1 1 0 1 1 1]', 115),
 ('[0 0 0 1 0 0 1 0]', 114),
 ('[1 0 1 0 1 0 1 0]', 109),
 ('[0 0 0 0 0 1 0 1]', 107),
 ('[0 1 0 1 1 1 1 1]', 106),
 ('[1 1 1 1 1 1 1 0]', 105),
 ('[0 1 1 1 1 1 1 1]', 105),
 ('[0 1 1 0 1 1 1 1]', 105),
 ('[1 1 0 1 1 1 1 1]', 103),
 ('[1 1 1